In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("titanic.csv")

In [3]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


In [5]:
columns_to_drop=['PassengerId','Name','Ticket','Cabin','Embarked']
data_clean=data.drop(columns_to_drop,axis=1)

In [6]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(4), object(1)
memory usage: 45.3+ KB


In [10]:
data_clean=data_clean.fillna(data_clean['Age'].mean())


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data_clean['Sex'] = le.fit_transform(data_clean['Sex'])

In [11]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [12]:
input_column=['Pclass','Sex','Age','SibSp','Parch','Fare']

In [13]:
output_column=['Survived']

In [14]:
x=data_clean[input_column]
y=data_clean[output_column]

In [15]:
def entropy(col):
    count=np.unique(col,return_counts=True)
    N=float(col.shape[0])
    ent=0
    for i in count[1]:
        p=i/N
        ent+=-1.0*p*np.log2(p)
    return ent

In [36]:

def split_data(X_data,fkey,fval):
    #Binary Split
    X_Right = pd.DataFrame([],columns=X_data.columns)
    X_Left = pd.DataFrame([],columns=X_data.columns)
    
    for ix in range(X_data.shape[0]):
        val = X_data[fkey][ix] 
        if val>fval:
            X_Right = X_Right.append(X_data.loc[ix])
        else:
            X_Left = X_Left.append(X_data.loc[ix])
            
    
    return X_Left,X_Right

In [37]:
def i_gain(x_data,fkey,fval):
    left,right=split_data(x_data,fkey,fval)
    
    #caluclating weight
    l=float(left.shape[0]/x_data.shape[0])
    r=float(right.shape[0]/x_data.shape[0])
    
    if left.shape[0]==0 or right.shape[0]==0:
        return-100000
    info_gai=entropy(x_data.Survived)-l*entropy(left.Survived)-r*entropy(right.Survived)
    return info_gai

In [40]:
class DecisionTree:
    def _init_(self,depth=0,max=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.depth=depth
        self.max=max
    def train(self,xtrain):
        feature=['Pclass','Sex','Age','SibSp','Parch','Fare']
        i_g=[]
        for ix in feature:
            inf=i_gain(xtrain,xtrain[ix],xtrain[ix].mean())
            i_g.append(inf)
        self.key=feature[np.argmax(i_g)]
        self.fval=self.key.mean()
        #split data
        data_left,data_right=split_data(xtrain,self.key,self.fval)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        if data_left.shape[0]==0 or data_right.shape==0:
            if x_train.survived.mean()>=0.5:
                self.target=0
            else:
                self.target=1
            return
        if (self.depth>=max_depth):
            if x_train.survived.mean()>=0.5:
                self.target=0
            else:
                self.target=1
            return
        #recursive
        self.left=DecisionTree(depth=self.depth+1,max=5)
        self.right.train(data_left)
        self.right=DecisionTree(depth=self.depth+1,max=5)
        self.right=train(data_right)
        if (self.depth>=max_depth):
            if x_train.survived.mean()>=0.5:
                self.target=0
            else:
                self.target=1
            return
        
        

In [41]:
d=DecisionTree()
d.train(data_clean)

KeyError: "None of [Int64Index([3, 1, 3, 1, 3, 3, 1, 3, 3, 2,\n            ...\n            3, 3, 2, 3, 3, 2, 1, 3, 1, 3],\n           dtype='int64', length=891)] are in the [columns]"